# Latest

In [ ]:
import json
import sys
import traceback
import pandas as pd
import re
import math
import cmath
import logging
import sympy
import multiprocessing
from sympy import symbols
from concurrent.futures import ProcessPoolExecutor, wait, ALL_COMPLETED

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def load_json_file(file_path):
    try:
        with open(file_path, 'r') as file:
            logging.info(f"Loading JSON file from: {file_path}")
            return json.load(file)
    except json.JSONDecodeError as e:
        logging.error(f"Error decoding JSON file: {e}")
        sys.exit(1)
    except FileNotFoundError:
        logging.error(f"File not found: {file_path}")
        sys.exit(1)
    except Exception as e:
        logging.error(f"An error occurred while reading the file: {e}")
        sys.exit(1)

def extract_function_name(python_code):
    match = re.search(r'def\s+([a-zA-Z_][a-zA-Z0-9_]*)\s*\(', python_code)
    if match:
        return match.group(1)
    logging.error("Function name could not be extracted from the code.")
    return None

def match_and_prepare_inputs(function_code, input_values):
    # Regex to extract parameter names from the function definition
    match = re.search(r'def\s+\w+\s*\((.*?)\):', function_code, re.DOTALL)
    if match:
        # Extract and clean the parameter names
        parameters = match.group(1).replace(" ", "").split(',')
        
        # Create a mapping of base names to possible suffix variations
        suffixes = ['_val', '_value']
        matched_inputs = {}
        
        for param in parameters:
            # Generate possible names with suffixes
            possible_names = [param]
            for suffix in suffixes:
                if param.endswith(suffix):
                    base_name = param[:-len(suffix)]
                    possible_names.append(base_name)
            
            # Match inputs with the base names and suffixes
            for name in possible_names:
                if name in input_values:
                    matched_inputs[param] = input_values[name]
                    break
            else:
                # If no match is found, set the parameter to None
                matched_inputs[param] = None
        
        logging.info(f"Matched inputs: {matched_inputs}")
        return matched_inputs
    logging.error("Failed to match and prepare inputs.")
    return {}





import sympy

def execute_python_code(code, input_values):
    try:
        function_name = extract_function_name(code)
        if not function_name:
            return  0#"No function found"

        prepared_inputs = match_and_prepare_inputs(code, input_values)

        code_namespace = {}
        exec(code, code_namespace)

        if function_name not in code_namespace:
            return f"Error: Function '{function_name}' not found in the generated code."

        # Execute the function
        result = code_namespace[function_name](**prepared_inputs)

        # Check if the result is a string
        if isinstance(result, str):
            # Try to convert the string to a SymPy expression
            try:
                sympy_expr = sympy.sympify(result)
                # Substitute values and evaluate
                for key, value in input_values.items():
                    sympy_expr = sympy_expr.subs(sympy.Symbol(key), value)
                result = sympy_expr.evalf()
            except sympy.SympifyError:
                # If we can't convert to a SymPy expression, return the original string
                return result

        # Handle symbolic results
        if isinstance(result, sympy.Basic):
            for key, value in input_values.items():
                if isinstance(value, (int, float)):
                    result = result.subs(sympy.Symbol(key), value)
            result = result.evalf()

        # Convert SymPy complex number to Python complex number
        if isinstance(result, sympy.Basic) and not result.is_real:
            real_part = float(result.as_real_imag()[0])
            imag_part = float(result.as_real_imag()[1])
            result = complex(real_part, imag_part)
        
        # Format complex numbers
        if isinstance(result, complex):
            real_part_str = f"{result.real}"
            imag_part_str = f"{result.imag}j"
            if result.imag >= 0:
                return f"{real_part_str}+{imag_part_str}"
            else:
                return f"{real_part_str}{imag_part_str}"
        if result in (float('inf'), math.inf, sympy.oo, sympy.S.Infinity):
            return "inf"
        elif result in (float('-inf'), -math.inf, -sympy.oo, sympy.S.NegativeInfinity):
            return "-inf"
        
        return str(result)

    except Exception as e:
        logging.error(f"An error occurred while executing the code: {e}")
        return 0 #f"An error occurred while executing the code: {e}"# General fallback



def process_task(row, json_data_list):
    task_id = row['id']
    python_code = row['generated_code']
    
    json_data = next((item for item in json_data_list if item['task_id'] == task_id), None)
    if json_data is None:
        logging.warning(f"No JSON data found for task_id: {task_id}")
        return task_id, []

    test_cases = json_data.get("test_cases", [])
    outputs = []
    for case in test_cases:
        input_values = case.get("input", {})
        result = execute_python_code(python_code, input_values)
        outputs.append(result)
    
    return task_id, outputs

def main(json_file, csv_file, output_file):
    try:
        json_data_list = load_json_file(json_file)
        df = pd.read_csv(csv_file)

        results_dict = {
            "id": [],
            "outputs": []
        }

        with ProcessPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:
            futures = [executor.submit(process_task, row, json_data_list) for _, row in df.iterrows()]
            done, _ = wait(futures, return_when=ALL_COMPLETED)

            for future in done:
                task_id, outputs = future.result()
                results_dict["id"].append(task_id)
                results_dict["outputs"].append(outputs)

            # Properly shut down the executor after all tasks are complete
            executor.shutdown(wait=True)

        output_df = pd.DataFrame(results_dict)
        output_df.to_csv(output_file, index=False)

        logging.info(f"Outputs have been written to {output_file}")

    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
        traceback.print_exc()

# File paths
json_file = 'public_test_new_no_sol_no_out.json'
csv_file = 'codes_26_2.csv'
output_file = 'results_v16.csv'

if __name__ == "__main__":
    main(json_file, csv_file, output_file)


2024-08-26 17:34:30,901 - INFO - Loading JSON file from: public_test_new_no_sol_no_out.json
2024-08-26 17:34:32,462 - INFO - Matched inputs: {'x': 0.29686280231110906, 'y': -1.882691694932987, 'z': 3.431982744644353}
2024-08-26 17:34:32,478 - INFO - Matched inputs: {'x_value': -7.064625092178776}
2024-08-26 17:34:32,490 - INFO - Matched inputs: {'R': 7.440682794714104, 'r': 5.0957264936844044}


24805.0185890389*R*r**2

2024-08-26 17:34:32,501 - INFO - Matched inputs: {'x': 8.412355115832945}


2024-08-26 17:34:32,509 - INFO - Matched inputs: {'R': 4.504088231490985, 'r': 2.2421454297576706}


24805.0185890389*R*r**2


...

2024-08-26 17:34:38,949 - INFO - Matched inputs: {'v': -7.870592463510963, 'w': -9.098604051837928, 'x': 0.19358772100261668, 'y': 1.4327245850325898, 'z': -9.332336890608365}
2024-08-26 17:34:38,871 - INFO - Matched inputs: {'x': 2.8316611826868225, 'y': 7.0132178637146145, 'z': 5.495917439700636, 'w': 3.4991853914434303}


4.60913271139952406251/10 - cos(1)


2024-08-26 17:34:38,954 - INFO - Matched inputs: {'x': 3.706608205951506}


2024-08-26 17:34:38,952 - INFO - Matched inputs: {'x': 2.6629278152909954, 'C1': 3.100808916338064}
2024-08-26 17:34:38,954 - INFO - Matched inputs: {'n_val': 2.4702541593032965, 's_val': 3.560077734282998}



365.691179935036

In [ ]:
import json
import sys
import traceback
import pandas as pd
import re
import math
import cmath
import logging
import sympy
import multiprocessing
from sympy import symbols
from concurrent.futures import ProcessPoolExecutor, wait, ALL_COMPLETED

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def load_json_file(file_path):
    try:
        with open(file_path, 'r') as file:
            logging.info(f"Loading JSON file from: {file_path}")
            return json.load(file)
    except json.JSONDecodeError as e:
        logging.error(f"Error decoding JSON file: {e}")
        sys.exit(1)
    except FileNotFoundError:
        logging.error(f"File not found: {file_path}")
        sys.exit(1)
    except Exception as e:
        logging.error(f"An error occurred while reading the file: {e}")
        sys.exit(1)

def extract_function_name(python_code):
    match = re.search(r'def\s+([a-zA-Z_][a-zA-Z0-9_]*)\s*\(', python_code)
    if match:
        return match.group(1)
    logging.error("Function name could not be extracted from the code.")
    return None

def match_and_prepare_inputs(function_code, input_values):
    # Regex to extract parameter names from the function definition
    match = re.search(r'def\s+\w+\s*\((.*?)\):', function_code, re.DOTALL)
    if match:
        # Extract and clean the parameter names
        parameters = match.group(1).replace(" ", "").split(',')
        
        # Create a mapping of base names to possible suffix variations
        suffixes = ['_val', '_value']
        matched_inputs = {}
        
        for param in parameters:
            # Generate possible names with suffixes
            possible_names = [param]
            for suffix in suffixes:
                if param.endswith(suffix):
                    base_name = param[:-len(suffix)]
                    possible_names.append(base_name)
            
            # Match inputs with the base names and suffixes
            for name in possible_names:
                if name in input_values:
                    matched_inputs[param] = input_values[name]
                    break
            else:
                # If no match is found, set the parameter to None
                matched_inputs[param] = None
        
        logging.info(f"Matched inputs: {matched_inputs}")
        return matched_inputs
    logging.error("Failed to match and prepare inputs.")
    return {}





def execute_python_code(code, input_values):
    try:
        function_name = extract_function_name(code)
        if not function_name:
            return 0  # "No function found"

        prepared_inputs = match_and_prepare_inputs(code, input_values)

        code_namespace = {}
        exec(code, code_namespace)

        if function_name not in code_namespace:
            return f"Error: Function '{function_name}' not found in the generated code."

        # Execute the function and handle possible domain errors
        result = code_namespace[function_name](**prepared_inputs)
        # Handle symbolic results
        if isinstance(result, sympy.Basic):
            for key, value in input_values.items():
                if isinstance(value, (int, float)):
                    result = result.subs(symbols(key), value)

        # Convert SymPy complex number to Python complex number after evalf
        if isinstance(result, sympy.Basic) and not result.is_real:
            result = result.evalf()  # Ensure full numerical evaluation
            real_part = float(result.as_real_imag()[0])
            imag_part = float(result.as_real_imag()[1])
            result = complex(real_part, imag_part)
        
        # Convert Python complex number to string with correct formatting
        if isinstance(result, complex):
            real_part_str = f"{result.real}"
            imag_part_str = f"{result.imag}j"
            if result.imag >= 0:
                return f"{real_part_str}+{imag_part_str}"
            else:
                return f"{real_part_str}{imag_part_str}"
        
        return str(result)

    except Exception as e:
        logging.error(f"An error occurred while executing the code: {e}")
        return 0  # General fallback



def process_task(row, json_data_list):
    task_id = row['id']
    python_code = row['generated_code']
    
    json_data = next((item for item in json_data_list if item['task_id'] == task_id), None)
    if json_data is None:
        logging.warning(f"No JSON data found for task_id: {task_id}")
        return task_id, []

    test_cases = json_data.get("test_cases", [])
    outputs = []
    for case in test_cases:
        input_values = case.get("input", {})
        result = execute_python_code(python_code, input_values)
        outputs.append(result)
    
    return task_id, outputs

def main(json_file, csv_file, output_file):
    try:
        json_data_list = load_json_file(json_file)
        df = pd.read_csv(csv_file)

        results_dict = {
            "id": [],
            "outputs": []
        }

        with ProcessPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:
            futures = [executor.submit(process_task, row, json_data_list) for _, row in df.iterrows()]
            done, _ = wait(futures, return_when=ALL_COMPLETED)

            for future in done:
                task_id, outputs = future.result()
                results_dict["id"].append(task_id)
                results_dict["outputs"].append(outputs)

            # Properly shut down the executor after all tasks are complete
            executor.shutdown(wait=True)

        output_df = pd.DataFrame(results_dict)
        output_df.to_csv(output_file, index=False)

        logging.info(f"Outputs have been written to {output_file}")

    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
        traceback.print_exc()

# File paths
json_file = 'public_test_new_no_sol_no_out.json'
csv_file = 'codes_26_2.csv'
output_file = 'results_v16.csv'

if __name__ == "__main__":
    main(json_file, csv_file, output_file)


2024-08-26 17:26:08,864 - INFO - Loading JSON file from: public_test_new_no_sol_no_out.json
2024-08-26 17:26:10,343 - INFO - Matched inputs: {'x': 0.29686280231110906, 'y': -1.882691694932987, 'z': 3.431982744644353}
2024-08-26 17:26:10,387 - INFO - Matched inputs: {'x_value': -7.064625092178776}
2024-08-26 17:26:10,398 - INFO - Matched inputs: {'R': 7.440682794714104, 'r': 5.0957264936844044}


6*x**5 + x**4 + 4*x**3 + 2*x**2 + 9*x + 9*y**4 + 9*y**3 + 10*y**2 + 9*y + z**5 + 6*z**4 + 7*z**3 + 8*z**2 + 2*z


2024-08-26 17:26:10,405 - INFO - Matched inputs: {'x': 8.412355115832945}


24805.0185890389*R*r**2


2024-08-26 17:26:10,412 - INFO - Matched inputs: {'R': 4.504088231490985, 'r': 2.2421454297576706}
2024-08-26 17:26:10,413 - INFO - Matched inputs: {'x': -8.319838193104632, 'y': -7.49310048950565, 'z': -7.424376685773753}
2024-08-26 17:26:10,411 - INFO - Matched inputs: {'v': 3, 'w': 4, 'x': 0, 'y': 4, 'z': 0}


...

7*w/(6*x + 2) + 7*x**2/(9*x + 4) + y/(8*y + 7) + 5*z**2/(2*w + 9) - 9.0

2024-08-26 17:26:15,371 - INFO - Matched inputs: {'v': 7.383762128910102, 'w': 5.141573933661475, 'z': 1.2764701939004963, 'x': 9.780533655116121, 'y': 5.941689125711734}
2024-08-26 17:26:15,370 - INFO - Matched inputs: {'x_val': 5.539105859164027, 'a_val': 0.21785021500893553, 'b_val': 2.2094424874275576, 'c_val': 6.414547659060235}


2024-08-26 17:26:15,371 - INFO - Matched inputs: {'x': 9.926037334469825, 'y': 5.5407322445324, 'z': 3.508341484005283}


1258.1769283696*log(928.748471191326*x) + 811.768767475923


2024-08-26 17:26:15,379 - INFO - Matched inputs: {'x': 1.112244747342479}


# working

In [ ]:
import json
import sys
import traceback
import pandas as pd
import re
import logging
import sympy
import multiprocessing
from sympy import symbols
from concurrent.futures import ProcessPoolExecutor, wait, ALL_COMPLETED

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def load_json_file(file_path):
    try:
        with open(file_path, 'r') as file:
            logging.info(f"Loading JSON file from: {file_path}")
            return json.load(file)
    except json.JSONDecodeError as e:
        logging.error(f"Error decoding JSON file: {e}")
        sys.exit(1)
    except FileNotFoundError:
        logging.error(f"File not found: {file_path}")
        sys.exit(1)
    except Exception as e:
        logging.error(f"An error occurred while reading the file: {e}")
        sys.exit(1)

def extract_function_name(python_code):
    match = re.search(r'def\s+([a-zA-Z_][a-zA-Z0-9_]*)\s*\(', python_code)
    if match:
        return match.group(1)
    logging.error("Function name could not be extracted from the code.")
    return None

def match_and_prepare_inputs(function_code, input_values):
    match = re.search(r'def\s+\w+\s*\((.*?)\):', function_code, re.DOTALL)
    if match:
        parameters = match.group(1).replace(" ", "").split(',')
        filtered_parameters = [key for key in input_values.keys() if not key.endswith('_val')]

        matched_inputs = {param: input_values.get(param, None) for param in filtered_parameters}
        logging.info(f"Matched inputs: {matched_inputs}")
        return matched_inputs
    logging.error("Failed to match and prepare inputs.")
    return {}

def execute_python_code(code, input_values):
    try:
        function_name = extract_function_name(code)
        if not function_name:
            return 0 #"No function found"

        prepared_inputs = match_and_prepare_inputs(code, input_values)

        code_namespace = {}
        exec(code, code_namespace)

        if function_name not in code_namespace:
            return f"Error: Function '{function_name}' not found in the generated code."
        
        result = code_namespace[function_name](**prepared_inputs)

          # Handle symbolic results
        if isinstance(result, sympy.Basic):
            for key, value in input_values.items():
                if isinstance(value, (int, float)):
                    result = result.subs(symbols(key), value)
            result = result.evalf()

        # Convert SymPy complex number to Python complex number after evalf
        if isinstance(result, sympy.Basic) and not result.is_real:
            result = result.evalf()  # Ensure full numerical evaluation
            real_part = float(result.as_real_imag()[0])
            imag_part = float(result.as_real_imag()[1])
            result = complex(real_part, imag_part)
        
        # Convert Python complex number to string with correct formatting
        if isinstance(result, complex):
            real_part_str = f"{result.real}"
            imag_part_str = f"{result.imag}j"
            if result.imag >= 0:
                return f"{real_part_str}+{imag_part_str}"
            else:
                return f"{real_part_str}{imag_part_str}"
        
        return str(result)
    
    except Exception as e:
        logging.error(f"An error occurred while executing the code: {e}")
        return 0 #f"An error occurred while executing the code: {e}"

def process_task(row, json_data_list):
    task_id = row['id']
    python_code = row['generated_code']
    
    json_data = next((item for item in json_data_list if item['task_id'] == task_id), None)
    if json_data is None:
        logging.warning(f"No JSON data found for task_id: {task_id}")
        return task_id, []

    test_cases = json_data.get("test_cases", [])
    outputs = []
    for case in test_cases:
        input_values = case.get("input", {})
        result = execute_python_code(python_code, input_values)
        outputs.append(result)
    
    return task_id, outputs

def main(json_file, csv_file, output_file):
    try:
        json_data_list = load_json_file(json_file)
        df = pd.read_csv(csv_file)

        results_dict = {
            "id": [],
            "outputs": []
        }

        with ProcessPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:
            futures = [executor.submit(process_task, row, json_data_list) for _, row in df.iterrows()]
            done, _ = wait(futures, return_when=ALL_COMPLETED)

            for future in done:
                task_id, outputs = future.result()
                results_dict["id"].append(task_id)
                results_dict["outputs"].append(outputs)

            # Properly shut down the executor after all tasks are complete
            executor.shutdown(wait=True)

        output_df = pd.DataFrame(results_dict)
        output_df.to_csv(output_file, index=False)

        logging.info(f"Outputs have been written to {output_file}")

    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
        traceback.print_exc()

# File paths
json_file = 'public_test_new_no_sol_no_out.json'
csv_file = 'codes_25_2.csv'
output_file = 'results_v12.csv'

if __name__ == "__main__":
    main(json_file, csv_file, output_file)


In [ ]:
import json
import sys
import traceback
import pandas as pd
import re
import logging
import sympy
import multiprocessing
import signal
from sympy import symbols
from concurrent.futures import ProcessPoolExecutor, as_completed, TimeoutError

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def load_json_file(file_path):
    try:
        with open(file_path, 'r') as file:
            logging.info(f"Loading JSON file from: {file_path}")
            return json.load(file, cls=json.JSONDecoder, object_hook=lambda d: {k: v for k, v in d.items()})
    except json.JSONDecodeError as e:
        logging.error(f"Error decoding JSON file: {e}")
        sys.exit(1)
    except FileNotFoundError:
        logging.error(f"File not found: {file_path}")
        sys.exit(1)
    except Exception as e:
        logging.error(f"An error occurred while reading the file: {e}")
        sys.exit(1)

def extract_function_name(python_code):
    match = re.search(r'def\s+([a-zA-Z_][a-zA-Z0-9_]*)\s*\(', python_code)
    if match:
        return match.group(1)
    logging.error("Function name could not be extracted from the code.")
    return None

def match_and_prepare_inputs(function_code, input_values):
    match = re.search(r'def\s+\w+\s*\((.*?)\):', function_code, re.DOTALL)
    if match:
        parameters = set(match.group(1).replace(" ", "").split(','))
        available_keys = set(input_values.keys())
        matched_inputs = {param: input_values[param] for param in parameters if param in available_keys}
        logging.info(f"Matched inputs: {matched_inputs}")
        return matched_inputs
    logging.error("Failed to match and prepare inputs.")
    return {}

def compile_python_code(code):
    compiled_code = compile(code, "<string>", "exec")
    code_namespace = {}
    exec(compiled_code, code_namespace)
    return code_namespace

def execute_python_code(code_namespace, function_name, input_values, timeout=10):
    try:
        result = None
        def handler(signum, frame):
            raise TimeoutError("Execution timed out")

        signal.signal(signal.SIGALRM, handler)
        signal.alarm(timeout)  # Set the timeout alarm
        
        if function_name in code_namespace:
            result = code_namespace[function_name](**input_values)
        
        # Cancel the alarm
        signal.alarm(0)

        # Simplified substitution step
        if isinstance(result, sympy.Basic):
            result = result.evalf(subs={symbols(k): v for k, v in input_values.items() if isinstance(v, (int, float))})

        # Handle complex results
        if isinstance(result, sympy.Basic) and not result.is_real:
            result = complex(result.as_real_imag()[0].evalf(), result.as_real_imag()[1].evalf())

        # Convert Python complex number to string with correct formatting
        if isinstance(result, complex):
            real_part_str = f"{result.real}"
            imag_part_str = f"{result.imag}j"
            if result.imag >= 0:
                return f"{real_part_str}+{imag_part_str}"
            else:
                return f"{real_part_str}{imag_part_str}"

        return str(result)

    except TimeoutError:
        logging.error("Execution timed out")
        return "Execution timed out"
    except Exception as e:
        logging.error(f"An error occurred while executing the code: {e}")
        return f"An error occurred while executing the code: {e}"

def process_task(row_dict, json_data_list):
    try:
        task_id = row_dict['id']
        python_code = row_dict['generated_code']

        json_data = next((item for item in json_data_list if item['task_id'] == task_id), None)
        if json_data is None:
            logging.warning(f"No JSON data found for task_id: {task_id}")
            return task_id, []

        test_cases = json_data.get("test_cases", [])
        code_namespace = compile_python_code(python_code)
        function_name = extract_function_name(python_code)

        if not function_name:
            return task_id, ["No function found"]

        outputs = []
        for case in test_cases:
            input_values = match_and_prepare_inputs(python_code, case.get("input", {}))
            result = execute_python_code(code_namespace, function_name, input_values)
            outputs.append(result)

        return task_id, outputs

    except Exception as e:
        logging.error(f"Error in process_task for task_id {row_dict.get('id', 'Unknown')}: {e}")
        return row_dict.get('id', 'Unknown'), [f"Error: {str(e)}"]

def batch_process_tasks(rows, json_data_list):
    results = []
    for _, row in rows.iterrows():
        results.append(process_task(row.to_dict(), json_data_list))
    return results

def main(json_file, csv_file, output_file):
    try:
        json_data_list = load_json_file(json_file)
        df = pd.read_csv(csv_file)

        results_dict = {
            "id": [],
            "outputs": []
        }

        # Test in single-process mode first to isolate issues
        single_process_mode = False

        if single_process_mode:
            logging.info("Running in single-process mode for debugging.")
            results = batch_process_tasks(df, json_data_list)
            for task_id, outputs in results:
                results_dict["id"].append(task_id)
                results_dict["outputs"].append(outputs)
        else:
            with ProcessPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:
                batch_size = 10  # Adjust batch size for your system
                rows_batches = [df.iloc[i:i+batch_size] for i in range(0, len(df), batch_size)]
                futures = [executor.submit(batch_process_tasks, batch, json_data_list) for batch in rows_batches]

                for future in as_completed(futures):
                    try:
                        results = future.result(timeout=600)  # Set an overall timeout for each batch
                        for task_id, outputs in results:
                            results_dict["id"].append(task_id)
                            results_dict["outputs"].append(outputs)
                    except TimeoutError:
                        logging.error("Batch processing timed out")
                    except Exception as e:
                        logging.error(f"An unexpected error occurred: {e}")
                        traceback.print_exc()

        output_df = pd.DataFrame(results_dict)
        output_df.to_csv(output_file, index=False)

        logging.info(f"Outputs have been written to {output_file}")

    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
        traceback.print_exc()

# File paths
json_file = 'public_test_new_no_sol_no_out.json'
csv_file = 'codes_25_3.csv'
output_file = 'results_v13.csv'

if __name__ == "__main__":
    main(json_file, csv_file, output_file)


2024-08-26 02:33:13,266 - INFO - Loading JSON file from: public_test_new_no_sol_no_out.json


83


2024-08-26 02:33:14,583 - INFO - Matched inputs: {'z': 8.213648132395043, 'x': 9.505062946262541, 'y': -0.5011321452467001}
2024-08-26 02:33:14,580 - INFO - Matched inputs: {'z': 3.431982744644353, 'x': 0.29686280231110906, 'y': -1.882691694932987}
2024-08-26 02:33:14,582 - INFO - Matched inputs: {'z': -7.424376685773753, 'x': -8.319838193104632, 'y': -7.49310048950565}
2024-08-26 02:33:14,584 - INFO - Matched inputs: {'z': -5.543633292104541, 'x': 2.2688166633745244, 'y': -4.093426980093597}


2024-08-26 02:33:14,585 - INFO - Matched inputs: {'z': -5.358424045484647, 'x': -8.483149816788513, 'y': 3.7519583070867686}


179

...

2024-08-26 02:33:22,150 - INFO - Matched inputs: {'w': -3.4475470014797116, 'z': 9.797275190162527, 'x': -8.877932212602062, 'y': 5.980058668042851}
2024-08-26 02:33:22,150 - INFO - Matched inputs: {'w': -8.507158835432357, 'z': -6.9211392225698525, 'x': 2.6441456170333506, 'y': -7.591811183039788}
2024-08-26 02:33:22,151 - INFO - Matched inputs: {}
2024-08-26 02:33:22,151 - INFO - Matched inputs: {}
2024-08-26 02:33:22,151 - INFO - Matched inputs: {'z': 5.733965697388781, 'x': -1.4680995691326864, 'y': 7.371183805557987}


2024-08-26 02:33:22,149 - ERROR - An error occurred while executing the code: Cannot convert complex to float
2024-08-26 02:33:22,148 - INFO - Matched inputs: {}
2024-08-26 02:33:22,150 - INFO - Matched inputs: {'z': 7.929476563954154, 'x': 7.010897999785303}
2024-08-26 02:33:22,148 - INFO - Matched inputs: {'y': -3.6660351110107037, 'R': 5.574534303128136, 'x': 3.043769237403964, 'r': 7.244555133344136, 'w': -6.918990818341837, 'z': -3.073940407256104}
2024-08-26 02:33:22,148 - INFO - Matched inputs: {'x': 3.1662849037744643, 'y': -2.4551494789539285}
2024-08-26 02:33:22,150 - INFO - Matched inputs: {'w': -4.751556597203246, 'z': 1.1368103106112812, 'x': -4.206714395064381, 'y': 4.857555515794484}


2024-08-26 02:33:22,153 - INFO - Matched inputs: {}
2024-08-26 02:33:22,153 - INFO - Matched inputs: {}
2024-08-26 02:33:22,156 - INFO - Matched inputs: {'w': -2.7384211484495484, 'z': 5.011678867223679, 'x': 1.765924336545119, 'y': -8.891137917679488}
2024-08-26 02:33:22,153 - ERROR - An error occurred while executing the code: calculate() missing 4 required positional arguments: 's_val', 'w_val', 'y_val', and 'z_val'
2024-08-26 02:33:22,155 - INFO - Matched inputs: {'y': -5.36336501349576, 'x': -3.9213514413682056, 'w': 6.741282720942859, 'z': 8.906336569621217, 'a': -1.3754630290534813}
2024-08-26 02:33:22,156 - INFO - Matched inputs: {'w': 7.282276805531826, 'z': -7.810850794195108, 'x': -9.21436380535448, 'y': -0.8244833780317293}
2024-08-26 02:33:22,154 - ERROR - An error occurred while executing the code: calculate() missing 2 required positional arguments: 'x_val' and 'y_val'
2024-08-26 02:33:22,151 - INFO - Matched inputs: {'x': -7.8030724916208705, 'y': 8.81337959308371}
2024

2024-08-26 02:33:22,248 - ERROR - An error occurred while executing the code: calculate() missing 3 required positional arguments: 'x_val', 'y_val', and 'z_val'
2024-08-26 02:33:22,248 - INFO - Matched inputs: {'w': 8.433614810502512, 'y': 5.960409522411528, 'x': 2.1875924860188825, 'r': 3.8639619986781764, 'h': 2.3306155064130873, 'z': -3.003023109132428}
2024-08-26 02:33:22,248 - INFO - Matched inputs: {'y': 2.542862551103248, 'x': 5.4195401881756595, 'w': 2.567331035353819, 'z': -7.368820625789185, 'v': -8.73255301761112}
2024-08-26 02:33:22,249 - ERROR - An error occurred while executing the code: calculate() missing 1 required positional argument: 'x_value'
2024-08-26 02:33:22,249 - INFO - Matched inputs: {}
2024-08-26 02:33:22,250 - INFO - Matched inputs: {'x': -9.165488973242756, 'y': 3.3790436066452774}
2024-08-26 02:33:22,249 - INFO - Matched inputs: {'z': 5.702038086305452, 'x': 4.2121969359718126, 'y': -4.213572261604906}
2024-08-26 02:33:22,252 - INFO - Matched inputs: {}
2

2024-08-26 02:33:22,254 - INFO - Matched inputs: {'w': 4.003509113460556, 'y': -5.438111933164686, 'x': -4.244110220841371, 'r': -3.104421127692449, 'h': 3.659678503678128, 'z': 6.496954329106288}
2024-08-26 02:33:22,254 - INFO - Matched inputs: {'x': 1.5007644478826982, 'y': 7.576171028775647}


# Complex Numbers


In [ ]:
import json
import sys
import traceback
import pandas as pd
import re
import logging
import sympy
import multiprocessing
from sympy import symbols
from concurrent.futures import ProcessPoolExecutor, wait, ALL_COMPLETED

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def load_json_file(file_path):
    try:
        with open(file_path, 'r') as file:
            logging.info(f"Loading JSON file from: {file_path}")
            return json.load(file)
    except json.JSONDecodeError as e:
        logging.error(f"Error decoding JSON file: {e}")
        sys.exit(1)
    except FileNotFoundError:
        logging.error(f"File not found: {file_path}")
        sys.exit(1)
    except Exception as e:
        logging.error(f"An error occurred while reading the file: {e}")
        sys.exit(1)

def extract_function_name(python_code):
    match = re.search(r'def\s+([a-zA-Z_][a-zA-Z0-9_]*)\s*\(', python_code)
    if match:
        return match.group(1)
    logging.error("Function name could not be extracted from the code.")
    return None

def match_and_prepare_inputs(function_code, input_values):
    match = re.search(r'def\s+\w+\s*\((.*?)\):', function_code, re.DOTALL)
    if match:
        parameters = match.group(1).replace(" ", "").split(',')
        
        
        filtered_parameters = [key for key in input_values.keys() if not key.endswith('_val')]

        # Prepare inputs only for the filtered parameters
        matched_inputs = {param: input_values.get(param, None) for param in filtered_parameters}
        logging.info(f"Matched inputs: {matched_inputs}")
        return matched_inputs
    logging.error("Failed to match and prepare inputs.")
    return {}


def execute_python_code(code, input_values):
    try:
        function_name = extract_function_name(code)
        if not function_name:
            return "No function found"

        prepared_inputs = match_and_prepare_inputs(code, input_values)

        code_namespace = {}
        exec(code, code_namespace)

        if function_name not in code_namespace:
            return f"Error: Function '{function_name}' not found in the generated code."
        
        result = code_namespace[function_name](**prepared_inputs)

        # Handle symbolic results
        if isinstance(result, sympy.Basic):
            for key, value in input_values.items():
                if isinstance(value, (int, float)):
                    result = result.subs(symbols(key), value)

        # Convert SymPy complex number to Python complex number after evalf
        if isinstance(result, sympy.Basic) and not result.is_real:
            real_part = float(result.as_real_imag()[0])
            imag_part = float(result.as_real_imag()[1])
            result = complex(real_part, imag_part)
        
        # Convert Python complex number to string with correct formatting
        if isinstance(result, complex):
            real_part_str = f"{result.real}"
            imag_part_str = f"{result.imag}j"
            if result.imag >= 0:
                return f"{real_part_str}+{imag_part_str}"
            else:
                return f"{real_part_str}{imag_part_str}"
        
        return str(result)
    
    except Exception as e:
        logging.error(f"An error occurred while executing the code: {e}")
        return f"An error occurred while executing the code: {e}"


def process_task(row, json_data_list):
    task_id = row['id']
    python_code = row['generated_code']
    
    json_data = next((item for item in json_data_list if item['task_id'] == task_id), None)
    if json_data is None:
        logging.warning(f"No JSON data found for task_id: {task_id}")
        return task_id, []

    test_cases = json_data.get("test_cases", [])
    outputs = []
    for case in test_cases:
        input_values = case.get("input", {})
        result = execute_python_code(python_code, input_values)
        outputs.append(result)
    
    return task_id, outputs

def main(json_file, csv_file, output_file):
    try:
        json_data_list = load_json_file(json_file)
        df = pd.read_csv(csv_file)

        results_dict = {
            "id": [],
            "outputs": []
        }

        with ProcessPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:
            futures = [executor.submit(process_task, row, json_data_list) for _, row in df.iterrows()]
            done, _ = wait(futures, return_when=ALL_COMPLETED)

            for future in done:
                task_id, outputs = future.result()
                results_dict["id"].append(task_id)
                results_dict["outputs"].append(outputs)

            # Properly shut down the executor after all tasks are complete
            executor.shutdown(wait=True)

        output_df = pd.DataFrame(results_dict)
        output_df.to_csv(output_file, index=False)

        logging.info(f"Outputs have been written to {output_file}")

    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
        traceback.print_exc()

# File paths
json_file = 'public_test_new_no_sol_no_out.json'
csv_file = 'codes_26.csv'
output_file = 'results_v13.csv'

if __name__ == "__main__":
    main(json_file, csv_file, output_file)


2024-08-26 06:17:45,867 - INFO - Loading JSON file from: public_test_new_no_sol_no_out.json
2024-08-26 06:17:47,443 - INFO - Matched inputs: {'x': 0.29686280231110906, 'y': -1.882691694932987, 'z': 3.431982744644353}


83


2024-08-26 06:17:47,450 - INFO - Matched inputs: {'x': -8.319838193104632, 'y': -7.49310048950565, 'z': -7.424376685773753}


83


2024-08-26 06:17:47,453 - INFO - Matched inputs: {'x': 9.505062946262541, 'y': -0.5011321452467001, 'z': 8.213648132395043}


...

6.0


2024-08-26 06:17:50,549 - INFO - Matched inputs: {'x': -0.6126252503228446, 'C1': 4.43412500226599}
2024-08-26 06:17:50,561 - INFO - Matched inputs: {'x': 4.68166544786857, 'a': 8.343980941459638, 'b': 7.604192956848665, 'c': 9.285375688227836}
2024-08-26 06:17:50,567 - INFO - Matched inputs: {'x': 4.045640061192886, 'y': -8.461639476512207, 'z': 1.9431956016088954}


32


2024-08-26 06:17:50,575 - INFO - Matched inputs: {'x': -1.4266557096617927, 'y': 0.32581715680492174, 'z': -8.545086796050043}


323.13913279204803e-17


In [ ]:
import json
import sys
import traceback
import pandas as pd
import ast
import sympy as sp
import textwrap

def load_json_file(file_path):
    try:
        with open(file_path, 'r') as file:
            return json.load(file)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON file: {e}")
        sys.exit(1)
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        sys.exit(1)
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        sys.exit(1)

def parse_function(code):
    tree = ast.parse(code)
    for node in ast.walk(tree):
        if isinstance(node, ast.FunctionDef):
            return node.name, [arg.arg for arg in node.args.args]
    return None, None

def create_wrapper(func_name, params, code):
    indented_code = textwrap.indent(code, '    ')
    param_mapping = ', '.join(f"{p}=input_values.get('{p}', input_values.get('{p[:-4]}', input_values.get('x')))" for p in params)
    wrapper_code = f"""
def wrapper(input_values):
{indented_code}
    return {func_name}({param_mapping})
"""
    return wrapper_code

def execute_python_code(code, input_values):
    try:
        # Parse the function name and parameters
        func_name, params = parse_function(code)
        
        if func_name is None:
            return "Error: Function not found in the generated code."
        
        # Create a wrapper function
        wrapper_code = create_wrapper(func_name, params, code)
        
        # Execute the wrapper code
        exec_globals = {'sp': sp}
        exec(wrapper_code, exec_globals)
        
        # Call the wrapper function with input values
        result = exec_globals['wrapper'](input_values)
        
        # Handle the result type
        if isinstance(result, str):
            result = eval(result)
        if isinstance(result, complex):
            return f"{result.real}+{result.imag}j"
        return str(result)
    except Exception as e:
        return str(e)

def main(json_file, csv_file, output_file):
    try:
        # Load the JSON data
        json_data_list = load_json_file(json_file)
        
        # Load the CSV file containing generated code
        df = pd.read_csv(csv_file)
        
        # Initialize a dictionary to store the final results
        results_dict = {
            "id": [],
            "outputs": []
        }
        
        # Iterate over each row in the CSV file
        for _, row in df.iterrows():
            task_id = row['id']
            python_code = row['generated_code']
            
            # Find the corresponding JSON object
            json_data = next((item for item in json_data_list if item['task_id'] == task_id), None)
            if json_data is None:
                print(f"No JSON data found for task_id: {task_id}")
                continue
            
            test_cases = json_data.get("test_cases", [])
            outputs = []
            
            for case in test_cases:
                input_values = case.get("input", {})
                # Modify the keys to add '_val' suffix
                modified_input_values = {f"{key}_val": value for key, value in input_values.items()}
                # Execute the Python code with modified input values
                result = execute_python_code(python_code, modified_input_values)
                outputs.append(result)
            
            # Store the results
            results_dict["id"].append(task_id)
            results_dict["outputs"].append(outputs)
        
        # Convert the results to a DataFrame
        output_df = pd.DataFrame(results_dict)
        
        # Write the DataFrame to a CSV file
        output_df.to_csv(output_file, index=False)
        print(f"Outputs have been written to {output_file}")
    
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        traceback.print_exc()

# File paths
json_file = 'public_test_new_no_sol_no_out.json'
csv_file = 'codes_25_2.csv'
output_file = 'results_v13.csv'

# Run the main function
if __name__ == "__main__":
    main(json_file, csv_file, output_file)

83
83
83
83
83
17
17
17
17
17
99220.0743561555
99220.0743561555
99220.0743561555
99220.0743561555
99220.0743561555
9
9
9
9
9
-146
-146
-146
-146
-146
17.6772686497683*B*h
17.6772686497683*B*h
17.6772686497683*B*h
17.6772686497683*B*h
17.6772686497683*B*h
61.0000000000000
61.0000000000000
61.0000000000000
61.0000000000000
61.0000000000000
8 + 2*exp(2)
8 + 2*exp(2)
8 + 2*exp(2)
8 + 2*exp(2)
8 + 2*exp(2)
10
10
10
10
10
51.0000000000000
51.0000000000000
51.0000000000000
51.0000000000000
51.0000000000000
44
44
44
44
44
260
260
260
260
260
0.39835164835165116
0.39835164835165116
0.39835164835165116
0.39835164835165116
0.39835164835165116
2*v + 4*w + 7*x + 7*y + 4*z + 4
2*v + 4*w + 7*x + 7*y + 4*z + 4
2*v + 4*w + 7*x + 7*y + 4*z + 4
2*v + 4*w + 7*x + 7*y + 4*z + 4
2*v + 4*w + 7*x + 7*y + 4*z + 4
-5025.00000000000
-5025.00000000000
-5025.00000000000
-5025.00000000000
-5025.00000000000
110
110
110
110
110
40.83180960710672
40.83180960710672
40.83180960710672
40.83180960710672
40.83180960710672


# New

In [ ]:
import json
import sys
import traceback
import pandas as pd
import re
import logging
import sympy
import multiprocessing
from sympy import symbols
from concurrent.futures import ProcessPoolExecutor, wait, ALL_COMPLETED

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def load_json_file(file_path):
    try:
        with open(file_path, 'r') as file:
            return json.load(file)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON file: {e}")
        sys.exit(1)
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        sys.exit(1)
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        sys.exit(1)

def extract_function_name(python_code):
    match = re.search(r'def\s+([a-zA-Z_][a-zA-Z0-9_]*)\s*\(', python_code)
    if match:
        return match.group(1)
    logging.error("Function name could not be extracted from the code.")
    return None

def match_and_prepare_inputs(function_code, input_values):
    match = re.search(r'def\s+\w+\s*\((.*?)\):', function_code, re.DOTALL)
    if match:
        parameters = match.group(1).replace(" ", "").split(',')
        
        filtered_parameters = [key for key in input_values.keys() if not key.endswith('_val')]

        # Prepare inputs only for the filtered parameters
        matched_inputs = {param: input_values.get(param, None) for param in filtered_parameters}
        logging.info(f"Matched inputs: {matched_inputs}")
        return matched_inputs
    logging.error("Failed to match and prepare inputs.")
    return {}

def execute_python_code(code, input_values):
    try:
        function_name = extract_function_name(code)
        if not function_name:
            logging.error("No function found in the code.")
            return "No function found"

        prepared_inputs = match_and_prepare_inputs(code, input_values)

        code_namespace = {}
        exec(code, code_namespace)

        if function_name not in code_namespace:
            logging.error(f"Function '{function_name}' not found in the generated code.")
            return f"Error: Function '{function_name}' not found in the generated code."
        
        result = code_namespace[function_name](**prepared_inputs)

        logging.info(f"Function execution result: {result}")

        # Handle symbolic results
        if isinstance(result, sympy.Basic):
            for key, value in input_values.items():
                if isinstance(value, (int, float)):
                    result = result.subs(symbols(key), value)

        # Convert SymPy complex number to Python complex number after evalf
        if isinstance(result, sympy.Basic) and not result.is_real:
            result = result.evalf()  # Ensure full numerical evaluation
            real_part = float(result.as_real_imag()[0])
            imag_part = float(result.as_real_imag()[1])
            result = complex(real_part, imag_part)
        
        # Convert Python complex number to string with correct formatting
        if isinstance(result, complex):
            real_part_str = f"{result.real}"
            imag_part_str = f"{result.imag}j"
            if result.imag >= 0:
                return f"{real_part_str}+{imag_part_str}"
            else:
                return f"{real_part_str}{imag_part_str}"
        
        if result is None:
            logging.warning("Function returned None.")
            return "Function returned None"
        
        return str(result)
    
    except Exception as e:
        logging.error(f"An error occurred while executing the code: {e}")
        return f"An error occurred while executing the code: {e}"

def process_task(row, json_data_list):
    task_id = row['id']
    python_code = row['generated_code']
    
    logging.info(f"Processing task {task_id}")
    
    try:
        json_data = next((item for item in json_data_list if item['task_id'] == task_id), None)
        if json_data is None:
            logging.warning(f"No JSON data found for task_id: {task_id}")
            return task_id, ["No JSON data found"]

        test_cases = json_data.get("test_cases", [])
        outputs = []
        for i, case in enumerate(test_cases):
            input_values = case.get("input", {})
            logging.info(f"Executing test case {i+1} for task {task_id}")
            result = execute_python_code(python_code, input_values)
            outputs.append(result)
        
        logging.info(f"Completed processing task {task_id}")
        return task_id, outputs
    except Exception as e:
        logging.error(f"Error processing task {task_id}: {str(e)}")
        return task_id, [f"Error processing task: {str(e)}"]

def main(json_file, csv_file, output_file):
    try:
        json_data_list = load_json_file(json_file)
        df = pd.read_csv(csv_file)

        results_dict = {
            "id": [],
            "outputs": []
        }

        with ProcessPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:
            futures = {executor.submit(process_task, row, json_data_list): row['id'] for _, row in df.iterrows()}
            
            for future in as_completed(futures):
                task_id = futures[future]
                try:
                    result_id, outputs = future.result()
                    results_dict["id"].append(result_id)
                    results_dict["outputs"].append(outputs)
                except Exception as e:
                    logging.error(f"Task {task_id} failed: {str(e)}")
                    results_dict["id"].append(task_id)
                    results_dict["outputs"].append([f"Task failed: {str(e)}"])

        output_df = pd.DataFrame(results_dict)
        output_df.to_csv(output_file, index=False)

        logging.info(f"Outputs have been written to {output_file}")

    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
        traceback.print_exc()

# File paths
json_file = 'public_test_new_no_sol_no_out.json'
csv_file = 'codes_25_3.csv'
output_file = 'results_v15.csv'

if __name__ == "__main__":
    main(json_file, csv_file, output_file)

In [ ]:
import json
import sys
import traceback
import pandas as pd
import re
import logging
import sympy
import multiprocessing
from sympy import symbols
from concurrent.futures import ProcessPoolExecutor, as_completed
from functools import lru_cache

# Setup logging
logging.basicConfig(level=logging.WARNING, format='%(asctime)s - %(levelname)s - %(message)s')

def load_json_file(file_path):
    try:
        with open(file_path, 'r') as file:
            return json.load(file)
    except Exception as e:
        logging.error(f"Error loading JSON file: {e}")
        sys.exit(1)

@lru_cache(maxsize=None)
def extract_function_name(python_code):
    match = re.search(r'def\s+([a-zA-Z_][a-zA-Z0-9_]*)\s*\(', python_code)
    return match.group(1) if match else None

@lru_cache(maxsize=None)
def match_and_prepare_inputs(function_code, input_tuple):
    match = re.search(r'def\s+\w+\s*\((.*?)\):', function_code, re.DOTALL)
    if match:
        parameters = match.group(1).replace(" ", "").split(',')
        input_values = dict(input_tuple)
        filtered_parameters = [key for key in input_values.keys() if not key.endswith('_val')]
        return {param: input_values.get(param, None) for param in filtered_parameters}
    return {}

def execute_python_code(code, input_values):
    try:
        function_name = extract_function_name(code)
        if not function_name:
            return "No function found"

        prepared_inputs = match_and_prepare_inputs(code, tuple(input_values.items()))

        # Compile the code
        compiled_code = compile(code, '<string>', 'exec')
        
        # Use the existing global namespace
        code_namespace = globals().copy()

        # Execute the compiled code
        exec(compiled_code, code_namespace)

        if function_name not in code_namespace:
            return f"Error: Function '{function_name}' not found in the generated code."
        
        result = code_namespace[function_name](**prepared_inputs)

        if isinstance(result, sympy.Basic):
            result = result.evalf(subs={symbols(k): v for k, v in input_values.items() if isinstance(v, (int, float))})

        if isinstance(result, complex):
            return f"{result.real}+{result.imag}j" if result.imag >= 0 else f"{result.real}{result.imag}j"
        
        return str(result) if result is not None else "Function returned None"
    
    except Exception as e:
        return f"An error occurred while executing the code: {e}"

def process_task(row, json_data):
    task_id = row['id']
    python_code = row['generated_code']
    
    try:
        json_data_item = next((item for item in json_data if item['task_id'] == task_id), None)
        if json_data_item is None:
            return task_id, ["No JSON data found"]

        test_cases = json_data_item.get("test_cases", [])
        outputs = [execute_python_code(python_code, case.get("input", {})) for case in test_cases]
        
        return task_id, outputs
    except Exception as e:
        return task_id, [f"Error processing task: {str(e)}"]

def main(json_file, csv_file, output_file):
    try:
        json_data = load_json_file(json_file)
        df = pd.read_csv(csv_file)

        results = []

        with ProcessPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:
            futures = [executor.submit(process_task, row, json_data) for _, row in df.iterrows()]
            
            for future in as_completed(futures):
                results.append(future.result())

        output_df = pd.DataFrame(results, columns=['id', 'outputs'])
        output_df.to_csv(output_file, index=False)

        logging.warning(f"Outputs have been written to {output_file}")

    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
        traceback.print_exc()

# File paths
json_file = 'public_test_new_no_sol_no_out.json'
csv_file = 'codes_25_2.csv'
output_file = 'results_v15.csv'

if __name__ == "__main__":
    main(json_file, csv_file, output_file)

83
83
83
83
83
17
17
17
17
17
9
9-146

-1469


-1469
-146

9-14617.6772686497683*B*h


17.6772686497683*B*h

17.6772686497683*B*h441010

4410

17.6772686497683*B*h4410


4410

44
17.6772686497683*B*h
260
260
2600.39835164835165116

260
0.39835164835165116260

0.39835164835165116
0.39835164835165116
0.39835164835165116
11061.0000000000000

11061.0000000000000

11061.0000000000000

11061.0000000000000

11061.0000000000000

40.8318096071067251.0000000000000

40.8318096071067251.0000000000000

40.8318096071067251.0000000000000

40.83180960710672
51.000000000000040.83180960710672

51.0000000000000
498 + 2*exp(2)

49
8 + 2*exp(2)49

498 + 2*exp(2)
49

88 + 2*exp(2)

88 + 2*exp(2)

8
8
2*v + 4*w + 7*x + 7*y + 4*z + 48

2*v + 4*w + 7*x + 7*y + 4*z + 4
-5025.00000000000
2*v + 4*w + 7*x + 7*y + 4*z + 4-5025.00000000000

-5025.000000000002*v + 4*w + 7*x + 7*y + 4*z + 4

1.7388413681282224
-5025.00000000000
1.73884136812822242*v + 4*w + 7*x + 7*y + 4*z + 4

1.7388413681282224-5025.00000000000

1.7

# New compile and exec

In [ ]:
import json
import sys
import traceback
import pandas as pd
import re
import logging
import sympy
import multiprocessing
from sympy import symbols
from concurrent.futures import ProcessPoolExecutor, as_completed
from functools import lru_cache

# Setup logging
logging.basicConfig(level=logging.WARNING, format='%(asctime)s - %(levelname)s - %(message)s')

def load_json_file(file_path):
    try:
        with open(file_path, 'r') as file:
            return json.load(file)
    except Exception as e:
        logging.error(f"Error loading JSON file: {e}")
        sys.exit(1)

@lru_cache(maxsize=None)
def extract_function_name(python_code):
    match = re.search(r'def\s+([a-zA-Z_][a-zA-Z0-9_]*)\s*\(', python_code)
    return match.group(1) if match else None

@lru_cache(maxsize=None)
def match_and_prepare_inputs(function_code, input_tuple):
    match = re.search(r'def\s+\w+\s*\((.*?)\):', function_code, re.DOTALL)
    if match:
        parameters = match.group(1).replace(" ", "").split(',')
        input_values = dict(input_tuple)
        #filtered_parameters = [key for key in input_values.keys() if not key.endswith('_val')]
        return {param: input_values.get(param, None) for param in input_values}
    return {}

def execute_python_code(code, input_values):
    try:
        function_name = extract_function_name(code)
        if not function_name:
            return "No function found"

        prepared_inputs = match_and_prepare_inputs(code, tuple(input_values.items()))

        # Compile the code
        compiled_code = compile(code, '<string>', 'exec')
        
        # Use the existing global namespace
        code_namespace = globals().copy()

        # Execute the compiled code
        exec(compiled_code, code_namespace)

        if function_name not in code_namespace:
            return f"Error: Function '{function_name}' not found in the generated code."
        
        result = code_namespace[function_name](**prepared_inputs)

        if isinstance(result, sympy.Basic):
            result = result.evalf(subs={symbols(k): v for k, v in input_values.items() if isinstance(v, (int, float))})

        if isinstance(result, complex):
            return f"{result.real}+{result.imag}j" if result.imag >= 0 else f"{result.real}{result.imag}j"
        
        return str(result) if result is not None else "Function returned None"
    
    except Exception as e:
        return f"An error occurred while executing the code: {e}"

def process_task(row, json_data):
    task_id = row['id']
    python_code = row['generated_code']
    
    try:
        json_data_item = next((item for item in json_data if item['task_id'] == task_id), None)
        if json_data_item is None:
            return task_id, ["No JSON data found"]

        test_cases = json_data_item.get("test_cases", [])
        outputs = [execute_python_code(python_code, case.get("input", {})) for case in test_cases]
        
        return task_id, outputs
    except Exception as e:
        return task_id, [f"Error processing task: {str(e)}"]

def main(json_file, csv_file, output_file):
    try:
        json_data = load_json_file(json_file)
        df = pd.read_csv(csv_file)

        results = []

        with ProcessPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:
            futures = [executor.submit(process_task, row, json_data) for _, row in df.iterrows()]
            
            for future in as_completed(futures):
                results.append(future.result())

        output_df = pd.DataFrame(results, columns=['id', 'outputs'])
        output_df.to_csv(output_file, index=False)

        logging.warning(f"Outputs have been written to {output_file}")

    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
        traceback.print_exc()

# File paths
json_file = 'public_test_new_no_sol_no_out.json'
csv_file = 'codes_26.csv'
output_file = 'results_v15.csv'

if __name__ == "__main__":
    main(json_file, csv_file, output_file)

83
83
83
83
83
2513.27384371161*pi**2*R*r**2
9.024805.0185890389

9.0
2513.27384371161*pi**2*R*r**29.0
17.6772686497683
-8624805.0185890389
9.0

17.6772686497683
2513.27384371161*pi**2*R*r**2-86
9.0


17.677268649768324805.0185890389-86


17.6772686497683-862513.27384371161*pi**2*R*r**2


17.6772686497683-8624805.0185890389


2002513.27384371161*pi**2*R*r**2

20024805.0185890389

200
200
200
51
51
51179

51179

51
179
179
0.8790322580645189179

0.8790322580645189
6310.7182818284590450.8790322580645189


0.87903225806451896310.718281828459045


0.879032258064518910.71828182845904563


10.71828182845904563

10.71828182845904563-6180


-6180
-6180
-6180
-6180-4035

-4035
-4035
-4035
-4035
40.83180960710672
40.83180960710672
40.8318096071067211.036383235143271

40.8318096071067211.036383235143271

4940.8318096071067211.036383235143271


4911.036383235143271

49
11.0363832351432714649


4946

46
46
46
1.7388413681282224
1.7388413681282224
1.7388413681282224
1.7388413681282224
1.738841368128

In [ ]:
! pip install groq datasets

In [ ]:
import os
from groq import Groq
from datasets import Dataset, DatasetDict, load_dataset
import json


from groq import Groq

client = Groq(
    api_key="yourapikey",
)

In [9]:
import json
import random
import sympy as sp
from tqdm import tqdm

# Template for generating synthetic data
SYNTHETIC_DATA_PROMPT_TEMPLATE = """\
Generate a LaTeX expression and its corresponding Python code (function) solution for the following equation type and complexity:

Equation type: {equation_type}
Complexity: {complexity} (1 = Easy, 2 = Medium-Easy, 3 = Medium, 4 = Medium-Hard, 5 = Hard)

The LaTeX expression should be syntactically correct and match the specified complexity level.
The Python code should use SymPy to define the expression.

Please return the response in the following format:

LaTeX: <LaTeX expression>
Python: <Python code>

Example:
LaTeX: \\pi h r^{{2}}  # LaTeX expression
Solution (Python Code):
import numpy as np
def geometric_function(r, h):
    return np.pi*h*r**2
"""


def generate_equation(client, equation_type):
    complexity = random.randint(1, 5)
    prompt = SYNTHETIC_DATA_PROMPT_TEMPLATE.format(equation_type=equation_type, complexity=complexity)

    try:
        response = client.chat.completions.create(
            model="llama-3.1-70b-versatile",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=2000
        )

        response_text = response.choices[0].message.content.strip()

        # Extract LaTeX and Python parts
        latex_part = response_text.split("LaTeX:")[1].split("Python:")[0].strip()
        python_part = response_text.split("Python:")[1].strip()

        return latex_part, python_part, complexity,response_text

    except Exception as e:
        print(f"Error generating data for {equation_type}: {str(e)}")
        return None, None, None,None




def generate_synthetic_data(client, equation_type):
    latex_expression, python_solution, complexity,complete_text = generate_equation(client, equation_type)

    if latex_expression is None or python_solution is None:
        return None

    # Generate a unique task_id
    task_id = f"task_{random.randint(1000, 9999)}"

    equation_type = equation_type
    # Structure the final data in the required format
    synthetic_data = {
        "task_id": task_id,
        "latex_expression": latex_expression,
        "solution": python_solution,
        "synthetic": True,
        "domain": "synthetic",
        "complexity": complexity,
        "equation_type": equation_type,
        "output_type": "numerical"
    }

    return synthetic_data

# ... (rest of the code remains the same)

def main(client):
    equation_types = [
        "Algebraic", "Multivariable Algebraic", "Trigonometric", "Logarithmic",
        "Exponential", "Exponential Decay", "Geometric", "Diophantine",
        "Summation", "Rational", "Fractional", "Derivative", "Integration",
        "Differential Equations"
    ]

    synthetic_data_list = []

    for eq_type in equation_types[11:]:
        for _ in tqdm(range(500), desc=f"Generating {eq_type} equations"):  # Generate 5 samples for each equation type
            synthetic_data = generate_synthetic_data(client, eq_type)
            if synthetic_data:
                synthetic_data_list.append(synthetic_data)

        # Convert the list of dictionaries to a JSON file
        with open("synthetic_data_v4.json", "w") as f:
            json.dump(synthetic_data_list, f, indent=4)

if __name__ == "__main__":

    main(client)

Generating Derivative equations:   0%|          | 2/500 [00:00<00:46, 10.68it/s]

Error generating data for Derivative: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-70b-versatile` in organization `org_01j5mdkg5hf75brm53tm0s2gz0` on : Limit 1000000, Used 1012262, Requested 165. Please try again in 17m53.7278s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}
Error generating data for Derivative: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-70b-versatile` in organization `org_01j5mdkg5hf75brm53tm0s2gz0` on : Limit 1000000, Used 1012261, Requested 165. Please try again in 17m53.6388s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}


Generating Derivative equations:   1%|          | 4/500 [00:00<00:47, 10.35it/s]

Error generating data for Derivative: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-70b-versatile` in organization `org_01j5mdkg5hf75brm53tm0s2gz0` on : Limit 1000000, Used 1012260, Requested 165. Please try again in 17m53.5308s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}
Error generating data for Derivative: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-70b-versatile` in organization `org_01j5mdkg5hf75brm53tm0s2gz0` on : Limit 1000000, Used 1012258, Requested 165. Please try again in 17m53.4268s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}
Error generating data for Derivative: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-70b-versatile` in organization `org_01j5mdkg5hf75brm53tm0s2gz0` on : Limit 1000000, Used 1012257, Requested 165. Please

Generating Derivative equations:   2%|▏         | 8/500 [00:00<00:46, 10.58it/s]

...

Error generating data for Derivative: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-70b-versatile` in organization `org_01j5mdkg5hf75brm53tm0s2gz0` on : Limit 1000000, Used 1012089, Requested 165. Please try again in 17m38.7848s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}
Error generating data for Derivative: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-70b-versatile` in organization `org_01j5mdkg5hf75brm53tm0s2gz0` on : Limit 1000000, Used 1012088, Requested 165. Please try again in 17m38.697799999s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}


Generating Derivative equations:  32%|███▏      | 159/500 [00:15<00:32, 10.42it/s]

Error generating data for Derivative: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-70b-versatile` in organization `org_01j5mdkg5hf75brm53tm0s2gz0` on : Limit 1000000, Used 1012087, Requested 165. Please try again in 17m38.5948s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}
Error generating data for Derivative: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-70b-versatile` in organization `org_01j5mdkg5hf75brm53tm0s2gz0` on : Limit 1000000, Used 1012085, Requested 165. Please try again in 17m38.4778s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}
Error generating data for Derivative: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-70b-versatile` in organization `org_01j5mdkg5hf75brm53tm0s2gz0` on : Limit 1000000, Used 1012084, Requested 165. Please

Generating Derivative equations:  32%|███▏      | 159/500 [00:15<00:33, 10.26it/s]


KeyboardInterrupt: 

In [8]:
import json
import random
import sympy as sp
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# Template for generating synthetic data
SYNTHETIC_DATA_PROMPT_TEMPLATE = """\
Generate a LaTeX expression and its corresponding Python code (function) solution for the following equation type and complexity:

Equation type: {equation_type}
Complexity: {complexity} (1 = Easy, 2 = Medium-Easy, 3 = Medium, 4 = Medium-Hard, 5 = Hard)

The LaTeX expression should be syntactically correct and match the specified complexity level.
The Python code should use SymPy to define the expression.

Please return the response in the following format:

LaTeX: <LaTeX expression>
Python: <Python code>

Example:
LaTeX: \\pi h r^{{2}}  # LaTeX expression
Solution (Python Code):
import numpy as np
def geometric_function(r, h):
    return np.pi*h*r**2
"""

def generate_equation(client, equation_type):
    complexity = random.randint(1, 5)
    prompt = SYNTHETIC_DATA_PROMPT_TEMPLATE.format(equation_type=equation_type, complexity=complexity)

    try:
        response = client.chat.completions.create(
            model="llama-3.1-70b-versatile",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=2000
        )

        response_text = response.choices[0].message.content.strip()

        # Extract LaTeX and Python parts
        latex_part = response_text.split("LaTeX:")[1].split("Python:")[0].strip()
        python_part = response_text.split("Python:")[1].strip()

        return latex_part, python_part, complexity, response_text

    except Exception as e:
        print(f"Error generating data for {equation_type}: {str(e)}")
        return None, None, None, None

def generate_synthetic_data(client, equation_type):
    latex_expression, python_solution, complexity, complete_text = generate_equation(client, equation_type)

    if latex_expression is None or python_solution is None:
        return None

    # Generate a unique task_id
    task_id = f"task_{random.randint(1000, 9999)}"

    # Structure the final data in the required format
    synthetic_data = {
        "task_id": task_id,
        "latex_expression": latex_expression,
        "solution": python_solution,
        "synthetic": True,
        "domain": "synthetic",
        "complexity": complexity,
        "equation_type": equation_type,
        "output_type": "numerical"
    }

    return synthetic_data

def main(client):
    equation_types = [
        "Algebraic", "Multivariable Algebraic", "Trigonometric", "Logarithmic",
        "Exponential", "Exponential Decay", "Geometric", "Diophantine",
        "Summation", "Rational", "Fractional", "Derivative", "Integration",
        "Differential Equations"
    ]

    synthetic_data_list = []

    with ThreadPoolExecutor() as executor:
        futures = []
        for eq_type in equation_types[11:]:
            for _ in range(500):  # Generate 500 samples for each equation type
                futures.append(executor.submit(generate_synthetic_data, client, eq_type))
            
            for future in tqdm(as_completed(futures), total=len(futures), desc=f"Generating {eq_type} equations"):
                synthetic_data = future.result()
                if synthetic_data:
                    synthetic_data_list.append(synthetic_data)

    # Convert the list of dictionaries to a JSON file
    with open("synthetic_data_v4.json", "w") as f:
        json.dump(synthetic_data_list, f, indent=4)

if __name__ == "__main__":
    main(client)


Generating Derivative equations:   1%|▏         | 7/500 [00:02<02:11,  3.75it/s]

Error generating data for Derivative: Error code: 503 - {'error': {'message': 'Service Unavailable', 'type': 'internal_server_error'}}
Error generating data for Derivative: Error code: 503 - {'error': {'message': 'Service Unavailable', 'type': 'internal_server_error'}}
Error generating data for Derivative: Error code: 503 - {'error': {'message': 'Service Unavailable', 'type': 'internal_server_error'}}
Error generating data for Derivative: Error code: 503 - {'error': {'message': 'Service Unavailable', 'type': 'internal_server_error'}}
Error generating data for Derivative: Error code: 503 - {'error': {'message': 'Service Unavailable', 'type': 'internal_server_error'}}
Error generating data for Derivative: Error code: 503 - {'error': {'message': 'Service Unavailable', 'type': 'internal_server_error'}}


Generating Derivative equations:   3%|▎         | 14/500 [00:03<01:07,  7.17it/s]

Error generating data for Derivative: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-70b-versatile` in organization `org_01j5mdkg5hf75brm53tm0s2gz0` on : Limit 1000000, Used 1001118, Requested 165. Please try again in 1m50.9336s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}
Error generating data for Derivative: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-70b-versatile` in organization `org_01j5mdkg5hf75brm53tm0s2gz0` on : Limit 1000000, Used 1001118, Requested 165. Please try again in 1m50.8726s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}
Error generating data for Derivative: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-70b-versatile` in organization `org_01j5mdkg5hf75brm53tm0s2gz0` on : Limit 1000000, Used 1001473, Requested 165. Please t

Generating Derivative equations:   5%|▌         | 27/500 [00:03<00:24, 19.17it/s]

...

Generating Integration equations:  55%|█████▍    | 549/1000 [00:01<00:00, 520.52it/s] 

Error generating data for Integration: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-70b-versatile` in organization `org_01j5mdkg5hf75brm53tm0s2gz0` on : Limit 1000000, Used 1012422, Requested 165. Please try again in 18m7.5698s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}
Error generating data for Integration: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-70b-versatile` in organization `org_01j5mdkg5hf75brm53tm0s2gz0` on : Limit 1000000, Used 1012422, Requested 165. Please try again in 18m7.5548s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}
Error generating data for Integration: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-70b-versatile` in organization `org_01j5mdkg5hf75brm53tm0s2gz0` on : Limit 1000000, Used 1012422, Requested 165. Pleas

Error generating data for Integration: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-70b-versatile` in organization `org_01j5mdkg5hf75brm53tm0s2gz0` on : Limit 1000000, Used 1012419, Requested 165. Please try again in 18m7.265799999s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}
Error generating data for Integration: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-70b-versatile` in organization `org_01j5mdkg5hf75brm53tm0s2gz0` on : Limit 1000000, Used 1012418, Requested 165. Please try again in 18m7.207799999s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}
Error generating data for Integration: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-70b-versatile` in organization `org_01j5mdkg5hf75brm53tm0s2gz0` on : Limit 1000000, Used 1012418, Requested 

KeyboardInterrupt: 

Error generating data for Integration: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-70b-versatile` in organization `org_01j5mdkg5hf75brm53tm0s2gz0` on : Limit 1000000, Used 1012299, Requested 165. Please try again in 17m56.9158s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}
Error generating data for Integration: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-70b-versatile` in organization `org_01j5mdkg5hf75brm53tm0s2gz0` on : Limit 1000000, Used 1012299, Requested 165. Please try again in 17m56.9148s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}
Error generating data for Integration: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-70b-versatile` in organization `org_01j5mdkg5hf75brm53tm0s2gz0` on : Limit 1000000, Used 1012297, Requested 165. Ple